## Assignment 2 Useful Concepts Walkthrough
This notebook covers the concepts that will be useful for solving the Assignment 2 problems:
1. Structured Outputs
3. Tavily
4. How to do RAG in LangGraph

In [ ]:
from dotenv import load_dotenv

load_dotenv()

import os
print(os.environ['TAVILY_API_KEY'][:20])
print(os.environ['OPENAI_API_KEY'][:20])


### Structured Outputs
Force the LLM to return the output in a specific format (Pydantic class) instead of just a single text. This is extremely useful for integrating LLMs into your applications in a seamless way.

Read more about that here - https://python.langchain.com/docs/concepts/structured_outputs/.

In [ ]:
from pydantic import BaseModel, Field
from langchain.chat_models import init_chat_model
from langchain_core.prompts import ChatPromptTemplate

model = init_chat_model("gpt-5-mini", model_provider="openai", reasoning_effort="minimal")

class ClassificationResult(BaseModel):
    category: str = Field(description="The category of the user question")
    explanation: str = Field(description="The explanation of why the category is chosen")

model = model.with_structured_output(ClassificationResult)
prompt = ChatPromptTemplate.from_template("""
Take the user question and classify it into one of the following categories:
Geography: Questions about countries, cities, or geographical features.
Science: Questions about science, technology, or scientific concepts.
Math: Questions about math, algebra, geometry, or mathematical concepts.
""")

chain = prompt | model
result = chain.invoke({"question": "What is the capital of France?"})
print(type(result))
print(result)

In [ ]:
# Without structured output, LLM will return only the text.
plain_model = init_chat_model("gpt-5-mini", model_provider="openai", reasoning_effort="minimal")
chain = prompt | plain_model
model_only_result = chain.invoke({"question": "What is the capital of France?"})
print(type(model_only_result))
print(model_only_result)

Structured output can also work with more complex data structures like lists as well. See the below example for identifying all places user asks in a question.

In [ ]:
from pydantic import BaseModel, Field

class Places(BaseModel):
    places: list[str] = Field(description="List of places")

model = init_chat_model("gpt-5-mini", model_provider="openai", reasoning_effort="minimal")

model = model.with_structured_output(Places)

# NOTE: This example is not answering the question, just extracting information
# from the question. You can also use it to answer the question.
prompt = ChatPromptTemplate.from_template("""
List all the places that the user asked in the question: {question}
""")

chain = prompt | model
result = chain.invoke({"question": "What is the capital of France and India?"})
print(type(result))
print(result)

## Tavily Web Search Example

In [ ]:
from langchain_tavily import TavilySearch

tool = TavilySearch(
    max_results=5,
    include_answer=True,  # True provides an answer from Tavily's side. Use False in assignments.
    include_raw_content=True,
    include_images=False,
    search_depth="advanced",  # advanced/basic are the two options
)

In [ ]:
# Calling the tool directly
tool.invoke({"query": "What are some latest innovations in AI?"})

In [ ]:
from typing import TypedDict
from langgraph.graph import StateGraph
from IPython.display import Image
from langgraph.graph import START, END

class WebSearchState(TypedDict):
    query: str
    results: dict
    char_counts: list[int]

def web_search(state: WebSearchState):
    results = tool.invoke({"query": state["query"]})
    return {"results": results}

# Counts the number of characters in the raw content of each result:
def sum_char_counts(state: WebSearchState):
    char_counts = [len(result['raw_content']) for result in state["results"]["results"]]
    return {"char_counts": char_counts}

graph_builder = StateGraph(WebSearchState)

graph_builder.add_node("web_search", web_search)
graph_builder.add_node("sum_char_counts", sum_char_counts)

graph_builder.add_edge(START, "web_search")
graph_builder.add_edge("web_search", "sum_char_counts")
graph_builder.add_edge("sum_char_counts", END)

graph = graph_builder.compile()

In [ ]:
# Display the graph
display(Image(graph.get_graph().draw_mermaid_png()))

In [ ]:
final_result = graph.invoke({"query": "What are some latest innovations in AI?"})
print(final_result['char_counts'])

## RAG Example in LangGraph

In [ ]:
# Example of loading a PDF file:
# Refer here for more examples of document loaders: https://python.langchain.com/docs/how_to/document_loader_pdf/
from langchain_community.document_loaders import PyPDFLoader

file_path = "/Users/aish/Downloads/2019-annual-performance-report.pdf"
loader = PyPDFLoader(file_path)
pages = []
async for page in loader.alazy_load():
    pages.append(page)

In [ ]:
print(len(pages))
test_page = pages[10]
print(type(test_page))
print(f"content: {test_page.page_content[:100]}")
print(f"metadata: {test_page.metadata}")
print(f"source: {test_page.metadata['source']}")


## Indexing the PDF file into a vector store

In [ ]:
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

vector_store = Chroma(
            embedding_function=embeddings,
            persist_directory="/Users/aish/chroma_db",
            collection_name="opm_documents_1"
        )

# Add the chunks (pages) from the PDF to the vector store
vector_store.add_documents(pages)

# Now perform similarity search
docs = vector_store.similarity_search("What are OPM's goals?", k=4)
print(len(docs))

In [ ]:
docs = vector_store.similarity_search("Who is the directory of OPM?", k=4)

## Retrieval in RAG

In [ ]:
from typing import TypedDict

# Create a retriever object on the vector store
retriever = vector_store.as_retriever()

from langchain.chat_models import init_chat_model
from langchain_core.prompts import PromptTemplate

model = init_chat_model("gpt-5-mini", model_provider="openai", reasoning_effort="minimal")

class GraphState(TypedDict):
    question: str
    documents: list[str]
    generated_answer: str

def retrieve_documents(state: GraphState):
    docs = vector_store.similarity_search(state["question"], k=4)
    return {"documents": docs}

def generate_answer(state: GraphState):
    # Call the model to generate the answer using the retrieved documents
    # and the question
    # Join all the retrieved documents into a single string
    prompt = PromptTemplate.from_template(
        "Answer the question based on the context provided. \n"
        "Question: {question}\n"
        "Context: {context}\n"
        "Answer: "
    )
    context = "\n".join([doc.page_content for doc in state["documents"]])
    prompt = prompt.format(question=state["question"], context=context)
    answer = model.invoke(prompt).content
    return {"generated_answer": answer}

graph_builder = StateGraph(GraphState)

graph_builder.add_node("retrieve_documents", retrieve_documents)
graph_builder.add_node("generate_answer", generate_answer)

graph_builder.add_edge(START, "retrieve_documents")
graph_builder.add_edge("retrieve_documents", "generate_answer")
graph_builder.add_edge("generate_answer", END)

graph = graph_builder.compile()

In [ ]:
# Display the graph
display(Image(graph.get_graph().draw_mermaid_png()))

In [ ]:
result = graph.invoke({"question": "What are OPM's goals?"})
print(result['generated_answer'])

In [ ]:
print(result['documents'])

In [ ]:
graph.invoke({"question": "Who won the baseball game today?"})